In [1]:
import pandas as pd

In [2]:
file = "data/marc_datenbank_170514.xlsx"

In [3]:
df = pd.read_excel(file)

In [4]:
df.fillna(value='False', inplace=True)

In [5]:
# create concept scheme:
# row['Bombertyp']
plane_type_scheme, _ = SkosConceptScheme.objects.get_or_create(
    dc_title='plane_type'
)
# row['Absturzursache']
reason_of_crash_scheme, _ = SkosConceptScheme.objects.get_or_create(
    dc_title='reason_of_crash'
)

In [6]:
for i, row in df.iterrows():
    bomb, _ = Bomber.objects.get_or_create(
        macr_nr=row['MACR-Nr']
    )
    sqr_wrt_name = "{}__{}__{}".format(
        row['Air Force'], row['Bomber Group'], row['Squadron']
    )
    sqd, _ = Institution.objects.get_or_create(
        legacy_id=sqr_wrt_name
    )
    sqd.written_name = "{} {} {}".format(
        row['Air Force'], row['Bomber Group'], row['Squadron']
    )
    sqd.save()
    bomb.squadron = sqd
    
    if row['Datum'] != 'False':
        bomb.date_of_crash = row['Datum']
    
    if row['Angriffsziel'] != 'False':
        target_place, _ = Place.objects.get_or_create(
            name=row['Angriffsziel']
        )
        bomb.target_place = target_place
    
    if row['Sichtung-Ort'] != 'False':
        last_seen, _ = Place.objects.get_or_create(
            name=row['Sichtung-Ort']
        )
        bomb.last_seen = last_seen

    if row['Absturzort'] != 'False':
        crash_place, _ = Place.objects.get_or_create(
            name=row['Absturzort']
        )
        bomb.crash_place = crash_place
    
    if row['Bombertyp'] != 'False':
        plane_type, _ = SkosConcept.objects.get_or_create(
            pref_label=row['Bombertyp']
        )
        plane_type.scheme.set([plane_type_scheme])
        bomb.plane_type = plane_type
    
    if row['Bombername'] != 'False':
        bomb.name = row['Bombername']
    
    if row['Absturzursache'] != 'False':
        reason_of_crash, _ = SkosConcept.objects.get_or_create(
            pref_label=row['Absturzursache']
        )
        reason_of_crash.scheme.set([reason_of_crash_scheme])
        bomb.reason_of_crash = reason_of_crash
     
    bomb.save()

In [7]:
#IdProvider.objects.all().delete()

In [8]:
# create concept scheme:
# row['Eintrag #']
eintrag_scheme, _ = SkosConceptScheme.objects.get_or_create(
    dc_title='eintrag'
)
# row['DG #']
dg_scheme, _ = SkosConceptScheme.objects.get_or_create(
    dc_title='dienstgrad'
)
# row['Erg #']
erg_scheme, _ = SkosConceptScheme.objects.get_or_create(
    dc_title='erg'
)

In [9]:
for x in range(1, 2):
    position = x
    for i, row in df.iterrows():
        bomb, _ = Bomber.objects.get_or_create(
            macr_nr=row['MACR-Nr']
        )
        name = "Name {}".format(position)
        dg = "DG {}".format(position)
        nr = "Nr {}".format(position)
        eintrag = "Eintrag {}".format(position)
        schicksal = "Schicksal {}".format(position)
        erg = "Erg {}".format(position)
        if row[nr] != 'False':
            pers, _ = Person.objects.get_or_create(
                dog_tag=row[nr]
            )
            pers.written_name =row[name]
            pers.part_of_bomber = bomb
            pers.save()

            if row[eintrag] != 'False':
                eintrag_c, _ = SkosConcept.objects.get_or_create(pref_label=row[eintrag])
                eintrag_c.scheme.set([eintrag_scheme])
                pers.destiny_stated = eintrag_c
                pers.save()

            if row[dg] != 'False':
                dg_c, _ = SkosConcept.objects.get_or_create(pref_label=row[dg])
                dg_c.scheme.set([dg_scheme])
                pers.rank = dg_c
                pers.save()

            if row[erg] != 'False':
                erg_c, _ = SkosConcept.objects.get_or_create(pref_label=row[erg])
                erg_c.scheme.set([erg_scheme])
                pers.destiny_checked = erg_c
                pers.save()